In [1]:
from weavepy import *
import xarray as xr
import seaborn as sns
sns.set_context("talk")

In [2]:
# --- Parameters for loading data ---
# It is expected that these will be changed rarely
## Variables
climate_variable = "TA"
energy_variable = "WON"
vars = [climate_variable, energy_variable]
techno = "30" # Techno needs to be specified for energy variables
## Temporal aggregation
aggregation_frequency = "D"
aggregation_function = "mean"
## Selection
country = "FR"
models = []
scenarios = []

In [3]:
data = load_vars([climate_variable, energy_variable], bdd_version = BDD_VERSION,
              countries = [country], technos = ["NA", techno], models = models, scenarios = scenarios, 
              aggregation_frequency = aggregation_frequency, aggregation_function = aggregation_function,
              verbose=True)

Loading country n°1/1 (FR)
|_Variable n°1/2 (TA)
|__Techno n°1/1 (NA)
|__ > File already exists, loading from cache: /Users/bourdin/Library/CloudStorage/OneDrive-Nexus365/WEAVE/PECD4.2/TA_NA_D-mean_FR_PECD4.2.nc
|_Variable n°2/2 (WON)
|__Techno n°1/1 (30)
|__ > File already exists, loading from cache: /Users/bourdin/Library/CloudStorage/OneDrive-Nexus365/WEAVE/PECD4.2/WON_30_D-mean_FR_PECD4.2.nc


In [4]:
# --- Period ---
# These can be changed and adjusted easily
historical_period = (1990, 2020)
future_period = (2070, 2100)

In [5]:
# --- Thresholds ---
# These can be changed and adjusted easily
comparisons = { # For each variable, choose among '<', '<=', '>', or '>='
    climate_variable:'<=', 
    energy_variable:'<',
}
thresholds = { # Number in units of variable
    climate_variable: 0,
    energy_variable: 0.30
}

In [6]:
# Identify problematic days
days = {}
for var in vars:
    days[var] = identify_pb_days(data[var], comparisons[var], thresholds[var])
days["compound"] = xr.concat(list(days.values()), dim = "var").all(dim = "var").rename("compound")
# Identify events
events = {}
for var in vars + ["compound"]:
    events[var] = identify_events_whole_base(days[var])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 329.67it/s]


## Number of events

In [7]:
event_count_barplot_multi(list(events.values()), historical_period, future_period)

FigureWidget({
    'data': [{'hovertemplate': 'historical (ERA5): %{y:.2f}<extra></extra>',
              'legendgroup': 'historical',
              'marker': {'color': 'rgb(0,0,0)'},
              'name': 'historical',
              'opacity': 0.5,
              'showlegend': True,
              'type': 'bar',
              'uid': 'f5a9a946-81e3-4815-a9fe-6fc4b0821559',
              'x': [historical],
              'xaxis': 'x',
              'y': [3.3421052631578947],
              'yaxis': 'y'},
             {'hovertemplate': 'Model: ERA5<br>Scenario: historical<br>Events: 3.34<extra></extra>',
              'legendgroup': 'historical',
              'marker': {'color': 'rgb(0,0,0)',
                         'line': {'color': 'black', 'width': 1},
                         'size': 10,
                         'symbol': 'triangle-down'},
              'mode': 'markers',
              'name': np.str_('ERA5'),
              'showlegend': False,
              'type': 'scatter',
        

In [8]:
nb_event_timeseries_multi(list(events.values()))

FigureWidget({
    'data': [{'customdata': [['SP126', 'AWI-_AWCM', 2015, 2035], ['SP126', 'AWI-
                             _AWCM', 2016, 2036], ['SP126', 'AWI-_AWCM', 2017,
                             2037], ['SP126', 'AWI-_AWCM', 2018, 2038], ['SP126',
                             'AWI-_AWCM', 2019, 2039], ['SP126', 'AWI-_AWCM', 2020,
                             2040], ['SP126', 'AWI-_AWCM', 2021, 2041], ['SP126',
                             'AWI-_AWCM', 2022, 2042], ['SP126', 'AWI-_AWCM', 2023,
                             2043], ['SP126', 'AWI-_AWCM', 2024, 2044], ['SP126',
                             'AWI-_AWCM', 2025, 2045], ['SP126', 'AWI-_AWCM', 2026,
                             2046], ['SP126', 'AWI-_AWCM', 2027, 2047], ['SP126',
                             'AWI-_AWCM', 2028, 2048], ['SP126', 'AWI-_AWCM', 2029,
                             2049], ['SP126', 'AWI-_AWCM', 2030, 2050], ['SP126',
                             'AWI-_AWCM', 2031, 2051], ['SP126', 'AWI-_AWCM', 2

## Duration 

In [9]:
event_duration_hist_multi(list(events.values()), historical_period, future_period)

FigureWidget({
    'data': [{'fill': 'tozeroy',
              'fillcolor': 'rgba(0,0,0,0.15)',
              'hovertemplate': ('Scenario: historical<br>Durati' ... 'rtion: %{y:.3%}<extra></extra>'),
              'legendgroup': 'historical',
              'legendrank': 0,
              'line': {'color': 'rgb(0,0,0)', 'width': 3},
              'mode': 'lines',
              'name': 'historical',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'd11794f4-0a34-4a08-a850-acbdc9af7a19',
              'x': {'bdata': ('AAAAAAAA8D8AAAAAAAAAQAAAAAAAAA' ... 'AAAAAgQAAAAAAAACBAAAAAAAAAIkA='),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('SvECmRQv0D9K8QKZFC/QP5Z7GmG5p9' ... 'iBTIqHP8ikeIFMiqc/yKR4gUyKpz8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'historical mean: 3.66 days<extra></extra>',
              'legendgroup': 'historical',
              'legend

## Seasonality

In [10]:
event_seasonality_kde_multi(list(days.values()), historical_period, future_period)

FigureWidget({
    'data': [{'customdata': {'bdata': ('AAAAAADQZkDsVwrU+kVnQNevFKj1u2' ... 'tXCuRlQBWo9SsFWmZAAAAAAADQZkA='),
                             'dtype': 'f8'},
              'hovertemplate': 'DOY: %{customdata:.0f}<br>Density: %{r:.3f}<extra></extra>',
              'legendgroup': np.str_('historical'),
              'line': {'color': 'rgb(0,0,0)'},
              'mode': 'lines',
              'name': np.str_('historical'),
              'r': {'bdata': ('jEhu6zPIBSvLGEgRz4oJKtmxAKp24x' ... 'hR2qEKLQMklM9yawYsjEhu6zPIBSs='),
                    'dtype': 'f8'},
              'subplot': 'polar',
              'theta': {'bdata': ('AAAAAACAZsCLLrroogtmwBdddNFFl2' ... 'TRRZdlQIwuuuiiC2ZAAAAAAACAZkA='),
                        'dtype': 'f8'},
              'type': 'scatterpolar',
              'uid': 'a50f14bb-9f05-41e9-bc55-1767b3746964'},
             {'customdata': {'bdata': ('AAAAAADQZkDsVwrU+kVnQNevFKj1u2' ... 'tXCuRlQBWo9SsFWmZAAAAAAADQZkA='),
                             'dt